coding: utf-8

# Optimization Methods<br>
<br>
Until now, you've always used Gradient Descent to update the parameters and minimize the cost. In this notebook, you will learn more advanced optimization methods that can speed up learning and perhaps even get you to a better final value for the cost function. Having a good optimization algorithm can be the difference between waiting days vs. just a few hours to get a good result. <br>
<br>
Gradient descent goes "downhill" on a cost function $J$. Think of it as trying to do this: <br>
<img src="images/cost.jpg" style="width:650px;height:300px;"><br>
<caption><center> <u> **Figure 1** </u>: **Minimizing the cost is like finding the lowest point in a hilly landscape**<br> At each step of the training, you update your parameters following a certain direction to try to get to the lowest possible point. </center></caption><br>
<br>
**Notations**: As usual, $\frac{\partial J}{\partial a } = $ `da` for any variable `a`.<br>
<br>
To get started, run the following code to import the libraries you will need.

### <font color='darkblue'> Updates to Assignment <font><br>
<br>
#### If you were working on a previous version<br>
* The current notebook filename is version "Optimization_methods_v1b". <br>
* You can find your work in the file directory as version "Optimization methods'.<br>
* To see the file directory, click on the Coursera logo at the top left of the notebook.<br>
<br>
#### List of Updates<br>
* op_utils is now opt_utils_v1a.  Assertion statement in `initialize_parameters` is fixed.<br>
* opt_utils_v1a: `compute_cost` function now accumulates total cost of the batch without taking the average (average is taken for entire epoch instead).<br>
* In `model` function, the total cost per mini-batch is accumulated, and the average of the entire epoch is taken as the average cost. So the plot of the cost function over time is now a smooth downward curve instead of an oscillating curve.<br>
* Print statements used to check each function are reformatted, and 'expected output` is reformatted to match the format of the print statements (for easier visual comparisons).

In[1]:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import math
import sklearn
import sklearn.datasets

In [ ]:
from opt_utils_v1a import load_params_and_grads, initialize_parameters, forward_propagation, backward_propagation
from opt_utils_v1a import compute_cost, predict, predict_dec, plot_decision_boundary, load_dataset
from testCases import *

In [ ]:
get_ipython().magic('matplotlib inline')
plt.rcParams['figure.figsize'] = (7.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

## 1 - Gradient Descent<br>
<br>
A simple optimization method in machine learning is gradient descent (GD). When you take gradient steps with respect to all $m$ examples on each step, it is also called Batch Gradient Descent. <br>
<br>
**Warm-up exercise**: Implement the gradient descent update rule. The  gradient descent rule is, for $l = 1, ..., L$: <br>
$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{1}$$<br>
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{2}$$<br>
<br>
where L is the number of layers and $\alpha$ is the learning rate. All parameters should be stored in the `parameters` dictionary. Note that the iterator `l` starts at 0 in the `for` loop while the first parameters are $W^{[1]}$ and $b^{[1]}$. You need to shift `l` to `l+1` when coding.

In[2]:

GRADED FUNCTION: update_parameters_with_gd

In [ ]:
def update_parameters_with_gd(parameters, grads, learning_rate):
    """
    Update parameters using one step of gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters to be updated:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients to update each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    learning_rate -- the learning rate, scalar.
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    L = len(parameters) // 2 # number of layers in the neural networks

    # Update rule for each parameter
    for l in range(L):
        ### START CODE HERE ### (approx. 2 lines)
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
        ### END CODE HERE ###
        
    return parameters

In[3]:

In [ ]:
parameters, grads, learning_rate = update_parameters_with_gd_test_case()

In [ ]:
parameters = update_parameters_with_gd(parameters, grads, learning_rate)
print("W1 =\n" + str(parameters["W1"]))
print("b1 =\n" + str(parameters["b1"]))
print("W2 =\n" + str(parameters["W2"]))
print("b2 =\n" + str(parameters["b2"]))

**Expected Output**:<br>
<br>
```<br>
W1 =<br>
[[ 1.63535156 -0.62320365 -0.53718766]<br>
 [-1.07799357  0.85639907 -2.29470142]]<br>
b1 =<br>
[[ 1.74604067]<br>
 [-0.75184921]]<br>
W2 =<br>
[[ 0.32171798 -0.25467393  1.46902454]<br>
 [-2.05617317 -0.31554548 -0.3756023 ]<br>
 [ 1.1404819  -1.09976462 -0.1612551 ]]<br>
b2 =<br>
[[-0.88020257]<br>
 [ 0.02561572]<br>
 [ 0.57539477]]<br>
```

A variant of this is Stochastic Gradient Descent (SGD), which is equivalent to mini-batch gradient descent where each mini-batch has just 1 example. The update rule that you have just implemented does not change. What changes is that you would be computing gradients on just one training example at a time, rather than on the whole training set. The code examples below illustrate the difference between stochastic gradient descent and (batch) gradient descent. <br>
<br>
- **(Batch) Gradient Descent**:<br>
<br>
``` python<br>
X = data_input<br>
Y = labels<br>
parameters = initialize_parameters(layers_dims)<br>
for i in range(0, num_iterations):<br>
    # Forward propagation<br>
    a, caches = forward_propagation(X, parameters)<br>
    # Compute cost.<br>
    cost += compute_cost(a, Y)<br>
    # Backward propagation.<br>
    grads = backward_propagation(a, caches, parameters)<br>
    # Update parameters.<br>
    parameters = update_parameters(parameters, grads)<br>
        <br>
```<br>
<br>
- **Stochastic Gradient Descent**:<br>
<br>
```python<br>
X = data_input<br>
Y = labels<br>
parameters = initialize_parameters(layers_dims)<br>
for i in range(0, num_iterations):<br>
    for j in range(0, m):<br>
        # Forward propagation<br>
        a, caches = forward_propagation(X[:,j], parameters)<br>
        # Compute cost<br>
        cost += compute_cost(a, Y[:,j])<br>
        # Backward propagation<br>
        grads = backward_propagation(a, caches, parameters)<br>
        # Update parameters.<br>
        parameters = update_parameters(parameters, grads)<br>
```<br>


In Stochastic Gradient Descent, you use only 1 training example before updating the gradients. When the training set is large, SGD can be faster. But the parameters will "oscillate" toward the minimum rather than converge smoothly. Here is an illustration of this: <br>
<br>
<img src="images/kiank_sgd.png" style="width:750px;height:250px;"><br>
<caption><center> <u> <font color='purple'> **Figure 1** </u><font color='purple'>  : **SGD vs GD**<br> "+" denotes a minimum of the cost. SGD leads to many oscillations to reach convergence. But each step is a lot faster to compute for SGD than for GD, as it uses only one training example (vs. the whole batch for GD). </center></caption><br>
<br>
**Note** also that implementing SGD requires 3 for-loops in total:<br>
1. Over the number of iterations<br>
2. Over the $m$ training examples<br>
3. Over the layers (to update all parameters, from $(W^{[1]},b^{[1]})$ to $(W^{[L]},b^{[L]})$)<br>
<br>
In practice, you'll often get faster results if you do not use neither the whole training set, nor only one training example, to perform each update. Mini-batch gradient descent uses an intermediate number of examples for each step. With mini-batch gradient descent, you loop over the mini-batches instead of looping over individual training examples.<br>
<br>
<img src="images/kiank_minibatch.png" style="width:750px;height:250px;"><br>
<caption><center> <u> <font color='purple'> **Figure 2** </u>: <font color='purple'>  **SGD vs Mini-Batch GD**<br> "+" denotes a minimum of the cost. Using mini-batches in your optimization algorithm often leads to faster optimization. </center></caption><br>
<br>
<font color='blue'><br>
**What you should remember**:<br>
- The difference between gradient descent, mini-batch gradient descent and stochastic gradient descent is the number of examples you use to perform one update step.<br>
- You have to tune a learning rate hyperparameter $\alpha$.<br>
- With a well-turned mini-batch size, usually it outperforms either gradient descent or stochastic gradient descent (particularly when the training set is large).

## 2 - Mini-Batch Gradient descent<br>
<br>
Let's learn how to build mini-batches from the training set (X, Y).<br>
<br>
There are two steps:<br>
- **Shuffle**: Create a shuffled version of the training set (X, Y) as shown below. Each column of X and Y represents a training example. Note that the random shuffling is done synchronously between X and Y. Such that after the shuffling the $i^{th}$ column of X is the example corresponding to the $i^{th}$ label in Y. The shuffling step ensures that examples will be split randomly into different mini-batches. <br>
<br>
<img src="images/kiank_shuffle.png" style="width:550px;height:300px;"><br>
<br>
- **Partition**: Partition the shuffled (X, Y) into mini-batches of size `mini_batch_size` (here 64). Note that the number of training examples is not always divisible by `mini_batch_size`. The last mini batch might be smaller, but you don't need to worry about this. When the final mini-batch is smaller than the full `mini_batch_size`, it will look like this: <br>
<br>
<img src="images/kiank_partition.png" style="width:550px;height:300px;"><br>
<br>
**Exercise**: Implement `random_mini_batches`. We coded the shuffling part for you. To help you with the partitioning step, we give you the following code that selects the indexes for the $1^{st}$ and $2^{nd}$ mini-batches:<br>
```python<br>
first_mini_batch_X = shuffled_X[:, 0 : mini_batch_size]<br>
second_mini_batch_X = shuffled_X[:, mini_batch_size : 2 * mini_batch_size]<br>
...<br>
```<br>
<br>
Note that the last mini-batch might end up smaller than `mini_batch_size=64`. Let $\lfloor s \rfloor$ represents $s$ rounded down to the nearest integer (this is `math.floor(s)` in Python). If the total number of examples is not a multiple of `mini_batch_size=64` then there will be $\lfloor \frac{m}{mini\_batch\_size}\rfloor$ mini-batches with a full 64 examples, and the number of examples in the final mini-batch will be ($m-mini_\_batch_\_size \times \lfloor \frac{m}{mini\_batch\_size}\rfloor$). 

In[4]:

GRADED FUNCTION: random_mini_batches

In [ ]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (1, number of examples)
    mini_batch_size -- size of the mini-batches, integer
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    np.random.seed(seed)            # To make your "random" minibatches the same as ours
    m = X.shape[1]                  # number of training examples
    mini_batches = []
        
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1,m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X = shuffled_X[:, k * mini_batch_size : (k+1) * mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : (k+1) * mini_batch_size]
        ### END CODE HERE ###
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        ### START CODE HERE ### (approx. 2 lines)
        mini_batch_X = shuffled_X[:, (mini_batch_size * num_complete_minibatches) : m]
        mini_batch_Y = shuffled_Y[:, (mini_batch_size * num_complete_minibatches) : m]
        ### END CODE HERE ###
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In[5]:

In [ ]:
X_assess, Y_assess, mini_batch_size = random_mini_batches_test_case()
mini_batches = random_mini_batches(X_assess, Y_assess, mini_batch_size)

In [ ]:
print ("shape of the 1st mini_batch_X: " + str(mini_batches[0][0].shape))
print ("shape of the 2nd mini_batch_X: " + str(mini_batches[1][0].shape))
print ("shape of the 3rd mini_batch_X: " + str(mini_batches[2][0].shape))
print ("shape of the 1st mini_batch_Y: " + str(mini_batches[0][1].shape))
print ("shape of the 2nd mini_batch_Y: " + str(mini_batches[1][1].shape)) 
print ("shape of the 3rd mini_batch_Y: " + str(mini_batches[2][1].shape))
print ("mini batch sanity check: " + str(mini_batches[0][0][0][0:3]))

**Expected Output**:<br>
<br>
<table style="width:50%"> <br>
    <tr><br>
    <td > **shape of the 1st mini_batch_X** </td> <br>
           <td > (12288, 64) </td> <br>
    </tr> <br>
    <br>
    <tr><br>
    <td > **shape of the 2nd mini_batch_X** </td> <br>
           <td > (12288, 64) </td> <br>
    </tr> <br>
    <br>
    <tr><br>
    <td > **shape of the 3rd mini_batch_X** </td> <br>
           <td > (12288, 20) </td> <br>
    </tr><br>
    <tr><br>
    <td > **shape of the 1st mini_batch_Y** </td> <br>
           <td > (1, 64) </td> <br>
    </tr> <br>
    <tr><br>
    <td > **shape of the 2nd mini_batch_Y** </td> <br>
           <td > (1, 64) </td> <br>
    </tr> <br>
    <tr><br>
    <td > **shape of the 3rd mini_batch_Y** </td> <br>
           <td > (1, 20) </td> <br>
    </tr> <br>
    <tr><br>
    <td > **mini batch sanity check** </td> <br>
           <td > [ 0.90085595 -0.7612069   0.2344157 ] </td> <br>
    </tr><br>
    <br>
</table>

<font color='blue'><br>
**What you should remember**:<br>
- Shuffling and Partitioning are the two steps required to build mini-batches<br>
- Powers of two are often chosen to be the mini-batch size, e.g., 16, 32, 64, 128.

## 3 - Momentum<br>
<br>
Because mini-batch gradient descent makes a parameter update after seeing just a subset of examples, the direction of the update has some variance, and so the path taken by mini-batch gradient descent will "oscillate" toward convergence. Using momentum can reduce these oscillations. <br>
<br>
Momentum takes into account the past gradients to smooth out the update. We will store the 'direction' of the previous gradients in the variable $v$. Formally, this will be the exponentially weighted average of the gradient on previous steps. You can also think of $v$ as the "velocity" of a ball rolling downhill, building up speed (and momentum) according to the direction of the gradient/slope of the hill. <br>
<br>
<img src="images/opt_momentum.png" style="width:400px;height:250px;"><br>
<caption><center> <u><font color='purple'>**Figure 3**</u><font color='purple'>: The red arrows shows the direction taken by one step of mini-batch gradient descent with momentum. The blue points show the direction of the gradient (with respect to the current mini-batch) on each step. Rather than just following the gradient, we let the gradient influence $v$ and then take a step in the direction of $v$.<br> <font color='black'> </center><br>
<br>
<br>
**Exercise**: Initialize the velocity. The velocity, $v$, is a python dictionary that needs to be initialized with arrays of zeros. Its keys are the same as those in the `grads` dictionary, that is:<br>
for $l =1,...,L$:<br>
```python<br>
v["dW" + str(l+1)] = ... #(numpy array of zeros with the same shape as parameters["W" + str(l+1)])<br>
v["db" + str(l+1)] = ... #(numpy array of zeros with the same shape as parameters["b" + str(l+1)])<br>
```<br>
**Note** that the iterator l starts at 0 in the for loop while the first parameters are v["dW1"] and v["db1"] (that's a "one" on the superscript). This is why we are shifting l to l+1 in the `for` loop.

In[12]:

GRADED FUNCTION: initialize_velocity

In [ ]:
def initialize_velocity(parameters):
    """
    Initializes the velocity as a python dictionary with:
                - keys: "dW1", "db1", ..., "dWL", "dbL" 
                - values: numpy arrays of zeros of the same shape as the corresponding gradients/parameters.
    Arguments:
    parameters -- python dictionary containing your parameters.
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    
    Returns:
    v -- python dictionary containing the current velocity.
                    v['dW' + str(l)] = velocity of dWl
                    v['db' + str(l)] = velocity of dbl
    """
    
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    
    # Initialize velocity
    for l in range(L):
        ### START CODE HERE ### (approx. 2 lines)
        v["dW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
        v["db" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
        ### END CODE HERE ###
        
    return v

In[13]:

In [ ]:
parameters = initialize_velocity_test_case()

In [ ]:
v = initialize_velocity(parameters)
print("v[\"dW1\"] =\n" + str(v["dW1"]))
print("v[\"db1\"] =\n" + str(v["db1"]))
print("v[\"dW2\"] =\n" + str(v["dW2"]))
print("v[\"db2\"] =\n" + str(v["db2"]))

**Expected Output**:<br>
<br>
```<br>
v["dW1"] =<br>
[[ 0.  0.  0.]<br>
 [ 0.  0.  0.]]<br>
v["db1"] =<br>
[[ 0.]<br>
 [ 0.]]<br>
v["dW2"] =<br>
[[ 0.  0.  0.]<br>
 [ 0.  0.  0.]<br>
 [ 0.  0.  0.]]<br>
v["db2"] =<br>
[[ 0.]<br>
 [ 0.]<br>
 [ 0.]]<br>
```

**Exercise**:  Now, implement the parameters update with momentum. The momentum update rule is, for $l = 1, ..., L$: <br>
<br>
$$ \begin{cases}<br>
v_{dW^{[l]}} = \beta v_{dW^{[l]}} + (1 - \beta) dW^{[l]} \\<br>
W^{[l]} = W^{[l]} - \alpha v_{dW^{[l]}}<br>
\end{cases}\tag{3}$$<br>
<br>
$$\begin{cases}<br>
v_{db^{[l]}} = \beta v_{db^{[l]}} + (1 - \beta) db^{[l]} \\<br>
b^{[l]} = b^{[l]} - \alpha v_{db^{[l]}} <br>
\end{cases}\tag{4}$$<br>
<br>
where L is the number of layers, $\beta$ is the momentum and $\alpha$ is the learning rate. All parameters should be stored in the `parameters` dictionary.  Note that the iterator `l` starts at 0 in the `for` loop while the first parameters are $W^{[1]}$ and $b^{[1]}$ (that's a "one" on the superscript). So you will need to shift `l` to `l+1` when coding.

In[14]:

GRADED FUNCTION: update_parameters_with_momentum

In [ ]:
def update_parameters_with_momentum(parameters, grads, v, beta, learning_rate):
    """
    Update parameters using Momentum
    
    Arguments:
    parameters -- python dictionary containing your parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v -- python dictionary containing the current velocity:
                    v['dW' + str(l)] = ...
                    v['db' + str(l)] = ...
    beta -- the momentum hyperparameter, scalar
    learning_rate -- the learning rate, scalar
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    v -- python dictionary containing your updated velocities
    """
    L = len(parameters) // 2 # number of layers in the neural networks
    
    # Momentum update for each parameter
    for l in range(L):
        
        ### START CODE HERE ### (approx. 4 lines)
        # compute velocities
        v["dW" + str(l+1)] = v["dW" + str(l+1)] * beta + ((1 - beta) *  grads['dW' + str(l+1)])
        v["db" + str(l+1)] = v["db" + str(l+1)] * beta + ((1 - beta) *  grads['db' + str(l+1)])
        # update parameters
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * v["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * v["db" + str(l+1)]
        ### END CODE HERE ###
        
    return parameters, v

In[15]:

In [ ]:
parameters, grads, v = update_parameters_with_momentum_test_case()

In [ ]:
parameters, v = update_parameters_with_momentum(parameters, grads, v, beta = 0.9, learning_rate = 0.01)
print("W1 = \n" + str(parameters["W1"]))
print("b1 = \n" + str(parameters["b1"]))
print("W2 = \n" + str(parameters["W2"]))
print("b2 = \n" + str(parameters["b2"]))
print("v[\"dW1\"] = \n" + str(v["dW1"]))
print("v[\"db1\"] = \n" + str(v["db1"]))
print("v[\"dW2\"] = \n" + str(v["dW2"]))
print("v[\"db2\"] = v" + str(v["db2"]))

**Expected Output**:<br>
<br>
```<br>
W1 = <br>
[[ 1.62544598 -0.61290114 -0.52907334]<br>
 [-1.07347112  0.86450677 -2.30085497]]<br>
b1 = <br>
[[ 1.74493465]<br>
 [-0.76027113]]<br>
W2 = <br>
[[ 0.31930698 -0.24990073  1.4627996 ]<br>
 [-2.05974396 -0.32173003 -0.38320915]<br>
 [ 1.13444069 -1.0998786  -0.1713109 ]]<br>
b2 = <br>
[[-0.87809283]<br>
 [ 0.04055394]<br>
 [ 0.58207317]]<br>
v["dW1"] = <br>
[[-0.11006192  0.11447237  0.09015907]<br>
 [ 0.05024943  0.09008559 -0.06837279]]<br>
v["db1"] = <br>
[[-0.01228902]<br>
 [-0.09357694]]<br>
v["dW2"] = <br>
[[-0.02678881  0.05303555 -0.06916608]<br>
 [-0.03967535 -0.06871727 -0.08452056]<br>
 [-0.06712461 -0.00126646 -0.11173103]]<br>
v["db2"] = v[[ 0.02344157]<br>
 [ 0.16598022]<br>
 [ 0.07420442]]<br>
```

**Note** that:<br>
- The velocity is initialized with zeros. So the algorithm will take a few iterations to "build up" velocity and start to take bigger steps.<br>
- If $\beta = 0$, then this just becomes standard gradient descent without momentum. <br>
<br>
**How do you choose $\beta$?**<br>
<br>
- The larger the momentum $\beta$ is, the smoother the update because the more we take the past gradients into account. But if $\beta$ is too big, it could also smooth out the updates too much. <br>
- Common values for $\beta$ range from 0.8 to 0.999. If you don't feel inclined to tune this, $\beta = 0.9$ is often a reasonable default. <br>
- Tuning the optimal $\beta$ for your model might need trying several values to see what works best in term of reducing the value of the cost function $J$. 

<font color='blue'><br>
**What you should remember**:<br>
- Momentum takes past gradients into account to smooth out the steps of gradient descent. It can be applied with batch gradient descent, mini-batch gradient descent or stochastic gradient descent.<br>
- You have to tune a momentum hyperparameter $\beta$ and a learning rate $\alpha$.

## 4 - Adam<br>
<br>
Adam is one of the most effective optimization algorithms for training neural networks. It combines ideas from RMSProp (described in lecture) and Momentum. <br>
<br>
**How does Adam work?**<br>
1. It calculates an exponentially weighted average of past gradients, and stores it in variables $v$ (before bias correction) and $v^{corrected}$ (with bias correction). <br>
2. It calculates an exponentially weighted average of the squares of the past gradients, and  stores it in variables $s$ (before bias correction) and $s^{corrected}$ (with bias correction). <br>
3. It updates parameters in a direction based on combining information from "1" and "2".<br>
<br>
The update rule is, for $l = 1, ..., L$: <br>
<br>
$$\begin{cases}<br>
v_{dW^{[l]}} = \beta_1 v_{dW^{[l]}} + (1 - \beta_1) \frac{\partial \mathcal{J} }{ \partial W^{[l]} } \\<br>
v^{corrected}_{dW^{[l]}} = \frac{v_{dW^{[l]}}}{1 - (\beta_1)^t} \\<br>
s_{dW^{[l]}} = \beta_2 s_{dW^{[l]}} + (1 - \beta_2) (\frac{\partial \mathcal{J} }{\partial W^{[l]} })^2 \\<br>
s^{corrected}_{dW^{[l]}} = \frac{s_{dW^{[l]}}}{1 - (\beta_2)^t} \\<br>
W^{[l]} = W^{[l]} - \alpha \frac{v^{corrected}_{dW^{[l]}}}{\sqrt{s^{corrected}_{dW^{[l]}}} + \varepsilon}<br>
\end{cases}$$<br>
where:<br>
- t counts the number of steps taken of Adam <br>
- L is the number of layers<br>
- $\beta_1$ and $\beta_2$ are hyperparameters that control the two exponentially weighted averages. <br>
- $\alpha$ is the learning rate<br>
- $\varepsilon$ is a very small number to avoid dividing by zero<br>
<br>
As usual, we will store all parameters in the `parameters` dictionary  

**Exercise**: Initialize the Adam variables $v, s$ which keep track of the past information.<br>
<br>
**Instruction**: The variables $v, s$ are python dictionaries that need to be initialized with arrays of zeros. Their keys are the same as for `grads`, that is:<br>
for $l = 1, ..., L$:<br>
```python<br>
v["dW" + str(l+1)] = ... #(numpy array of zeros with the same shape as parameters["W" + str(l+1)])<br>
v["db" + str(l+1)] = ... #(numpy array of zeros with the same shape as parameters["b" + str(l+1)])<br>
s["dW" + str(l+1)] = ... #(numpy array of zeros with the same shape as parameters["W" + str(l+1)])<br>
s["db" + str(l+1)] = ... #(numpy array of zeros with the same shape as parameters["b" + str(l+1)])<br>
<br>
```

In[18]:

GRADED FUNCTION: initialize_adam

In [ ]:
def initialize_adam(parameters) :
    """
    Initializes v and s as two python dictionaries with:
                - keys: "dW1", "db1", ..., "dWL", "dbL" 
                - values: numpy arrays of zeros of the same shape as the corresponding gradients/parameters.
    
    Arguments:
    parameters -- python dictionary containing your parameters.
                    parameters["W" + str(l)] = Wl
                    parameters["b" + str(l)] = bl
    
    Returns: 
    v -- python dictionary that will contain the exponentially weighted average of the gradient.
                    v["dW" + str(l)] = ...
                    v["db" + str(l)] = ...
    s -- python dictionary that will contain the exponentially weighted average of the squared gradient.
                    s["dW" + str(l)] = ...
                    s["db" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural networks
    v = {}
    s = {}
    
    # Initialize v, s. Input: "parameters". Outputs: "v, s".
    for l in range(L):
    ### START CODE HERE ### (approx. 4 lines)
        v["dW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
        v["db" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
        s["dW" + str(l+1)] = np.zeros(parameters["W" + str(l+1)].shape)
        s["db" + str(l+1)] = np.zeros(parameters["b" + str(l+1)].shape)
    ### END CODE HERE ###
    
    return v, s

In[19]:

In [ ]:
parameters = initialize_adam_test_case()

In [ ]:
v, s = initialize_adam(parameters)
print("v[\"dW1\"] = \n" + str(v["dW1"]))
print("v[\"db1\"] = \n" + str(v["db1"]))
print("v[\"dW2\"] = \n" + str(v["dW2"]))
print("v[\"db2\"] = \n" + str(v["db2"]))
print("s[\"dW1\"] = \n" + str(s["dW1"]))
print("s[\"db1\"] = \n" + str(s["db1"]))
print("s[\"dW2\"] = \n" + str(s["dW2"]))
print("s[\"db2\"] = \n" + str(s["db2"]))

**Expected Output**:<br>
<br>
```<br>
v["dW1"] = <br>
[[ 0.  0.  0.]<br>
 [ 0.  0.  0.]]<br>
v["db1"] = <br>
[[ 0.]<br>
 [ 0.]]<br>
v["dW2"] = <br>
[[ 0.  0.  0.]<br>
 [ 0.  0.  0.]<br>
 [ 0.  0.  0.]]<br>
v["db2"] = <br>
[[ 0.]<br>
 [ 0.]<br>
 [ 0.]]<br>
s["dW1"] = <br>
[[ 0.  0.  0.]<br>
 [ 0.  0.  0.]]<br>
s["db1"] = <br>
[[ 0.]<br>
 [ 0.]]<br>
s["dW2"] = <br>
[[ 0.  0.  0.]<br>
 [ 0.  0.  0.]<br>
 [ 0.  0.  0.]]<br>
s["db2"] = <br>
[[ 0.]<br>
 [ 0.]<br>
 [ 0.]]<br>
```

**Exercise**:  Now, implement the parameters update with Adam. Recall the general update rule is, for $l = 1, ..., L$: <br>
<br>
$$\begin{cases}<br>
v_{W^{[l]}} = \beta_1 v_{W^{[l]}} + (1 - \beta_1) \frac{\partial J }{ \partial W^{[l]} } \\<br>
v^{corrected}_{W^{[l]}} = \frac{v_{W^{[l]}}}{1 - (\beta_1)^t} \\<br>
s_{W^{[l]}} = \beta_2 s_{W^{[l]}} + (1 - \beta_2) (\frac{\partial J }{\partial W^{[l]} })^2 \\<br>
s^{corrected}_{W^{[l]}} = \frac{s_{W^{[l]}}}{1 - (\beta_2)^t} \\<br>
W^{[l]} = W^{[l]} - \alpha \frac{v^{corrected}_{W^{[l]}}}{\sqrt{s^{corrected}_{W^{[l]}}}+\varepsilon}<br>
\end{cases}$$<br>
<br>
<br>
**Note** that the iterator `l` starts at 0 in the `for` loop while the first parameters are $W^{[1]}$ and $b^{[1]}$. You need to shift `l` to `l+1` when coding.

In[20]:

GRADED FUNCTION: update_parameters_with_adam

In [ ]:
def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
    """
    Update parameters using Adam
    
    Arguments:
    parameters -- python dictionary containing your parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    learning_rate -- the learning rate, scalar.
    beta1 -- Exponential decay hyperparameter for the first moment estimates 
    beta2 -- Exponential decay hyperparameter for the second moment estimates 
    epsilon -- hyperparameter preventing division by zero in Adam updates
    Returns:
    parameters -- python dictionary containing your updated parameters 
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    """
    
    L = len(parameters) // 2                 # number of layers in the neural networks
    v_corrected = {}                         # Initializing first moment estimate, python dictionary
    s_corrected = {}                         # Initializing second moment estimate, python dictionary
    
    # Perform Adam update on all parameters
    for l in range(L):
        # Moving average of the gradients. Inputs: "v, grads, beta1". Output: "v".
        ### START CODE HERE ### (approx. 2 lines)
        v["dW" + str(l+1)] = v["dW" + str(l+1)] * beta1 + ((1 - beta1) *  grads['dW' + str(l+1)])
        v["db" + str(l+1)] = v["db" + str(l+1)] * beta1 + ((1 - beta1) *  grads['db' + str(l+1)])
        ### END CODE HERE ###

        # Compute bias-corrected first moment estimate. Inputs: "v, beta1, t". Output: "v_corrected".
        ### START CODE HERE ### (approx. 2 lines)
        v_corrected["dW" + str(l+1)] = v["dW" + str(l+1)] / (1 - np.power(beta1, t))
        v_corrected["db" + str(l+1)] = v["db" + str(l+1)] / (1 - np.power(beta1, t))
        ### END CODE HERE ###

        # Moving average of the squared gradients. Inputs: "s, grads, beta2". Output: "s".
        ### START CODE HERE ### (approx. 2 lines)
        s["dW" + str(l+1)] = s["dW" + str(l+1)] * beta2 + ((1 - beta2) *  np.power(grads['dW' + str(l+1)],2))
        s["db" + str(l+1)] = s["db" + str(l+1)] * beta2 + ((1 - beta2) *  np.power(grads['db' + str(l+1)],2))
        ### END CODE HERE ###

        # Compute bias-corrected second raw moment estimate. Inputs: "s, beta2, t". Output: "s_corrected".
        ### START CODE HERE ### (approx. 2 lines)
        s_corrected["dW" + str(l+1)] = s["dW" + str(l+1)] / (1 - np.power(beta2, t))
        s_corrected["db" + str(l+1)] = s["db" + str(l+1)] / (1 - np.power(beta2, t))
        ### END CODE HERE ###

        # Update parameters. Inputs: "parameters, learning_rate, v_corrected, s_corrected, epsilon". Output: "parameters".
        ### START CODE HERE ### (approx. 2 lines)
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - (learning_rate * v_corrected["dW" + str(l+1)] / (np.sqrt(s_corrected["dW" + str(l+1)]))+epsilon)
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - (learning_rate * v_corrected["db" + str(l+1)] / (np.sqrt(s_corrected["db" + str(l+1)]))+epsilon)
        ### END CODE HERE ###
    return parameters, v, s

In[21]:

In [ ]:
parameters, grads, v, s = update_parameters_with_adam_test_case()
parameters, v, s  = update_parameters_with_adam(parameters, grads, v, s, t = 2)

In [ ]:
print("W1 = \n" + str(parameters["W1"]))
print("b1 = \n" + str(parameters["b1"]))
print("W2 = \n" + str(parameters["W2"]))
print("b2 = \n" + str(parameters["b2"]))
print("v[\"dW1\"] = \n" + str(v["dW1"]))
print("v[\"db1\"] = \n" + str(v["db1"]))
print("v[\"dW2\"] = \n" + str(v["dW2"]))
print("v[\"db2\"] = \n" + str(v["db2"]))
print("s[\"dW1\"] = \n" + str(s["dW1"]))
print("s[\"db1\"] = \n" + str(s["db1"]))
print("s[\"dW2\"] = \n" + str(s["dW2"]))
print("s[\"db2\"] = \n" + str(s["db2"]))

**Expected Output**:<br>
<br>
```<br>
W1 = <br>
[[ 1.63178673 -0.61919778 -0.53561312]<br>
 [-1.08040999  0.85796626 -2.29409733]]<br>
b1 = <br>
[[ 1.75225313]<br>
 [-0.75376553]]<br>
W2 = <br>
[[ 0.32648046 -0.25681174  1.46954931]<br>
 [-2.05269934 -0.31497584 -0.37661299]<br>
 [ 1.14121081 -1.09245036 -0.16498684]]<br>
b2 = <br>
[[-0.88529978]<br>
 [ 0.03477238]<br>
 [ 0.57537385]]<br>
v["dW1"] = <br>
[[-0.11006192  0.11447237  0.09015907]<br>
 [ 0.05024943  0.09008559 -0.06837279]]<br>
v["db1"] = <br>
[[-0.01228902]<br>
 [-0.09357694]]<br>
v["dW2"] = <br>
[[-0.02678881  0.05303555 -0.06916608]<br>
 [-0.03967535 -0.06871727 -0.08452056]<br>
 [-0.06712461 -0.00126646 -0.11173103]]<br>
v["db2"] = <br>
[[ 0.02344157]<br>
 [ 0.16598022]<br>
 [ 0.07420442]]<br>
s["dW1"] = <br>
[[ 0.00121136  0.00131039  0.00081287]<br>
 [ 0.0002525   0.00081154  0.00046748]]<br>
s["db1"] = <br>
[[  1.51020075e-05]<br>
 [  8.75664434e-04]]<br>
s["dW2"] = <br>
[[  7.17640232e-05   2.81276921e-04   4.78394595e-04]<br>
 [  1.57413361e-04   4.72206320e-04   7.14372576e-04]<br>
 [  4.50571368e-04   1.60392066e-07   1.24838242e-03]]<br>
s["db2"] = <br>
[[  5.49507194e-05]<br>
 [  2.75494327e-03]<br>
 [  5.50629536e-04]]<br>
```

You now have three working optimization algorithms (mini-batch gradient descent, Momentum, Adam). Let's implement a model with each of these optimizers and observe the difference.

## 5 - Model with different optimization algorithms<br>
<br>
Lets use the following "moons" dataset to test the different optimization methods. (The dataset is named "moons" because the data from each of the two classes looks a bit like a crescent-shaped moon.) 

In[22]:

In [ ]:
train_X, train_Y = load_dataset()

We have already implemented a 3-layer neural network. You will train it with: <br>
- Mini-batch **Gradient Descent**: it will call your function:<br>
    - `update_parameters_with_gd()`<br>
- Mini-batch **Momentum**: it will call your functions:<br>
    - `initialize_velocity()` and `update_parameters_with_momentum()`<br>
- Mini-batch **Adam**: it will call your functions:<br>
    - `initialize_adam()` and `update_parameters_with_adam()`

In[27]:

In [ ]:
def model(X, Y, layers_dims, optimizer, learning_rate = 0.0007, mini_batch_size = 64, beta = 0.9,
          beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8, num_epochs = 10000, print_cost = True):
    """
    3-layer neural network model which can be run in different optimizer modes.
    
    Arguments:
    X -- input data, of shape (2, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (1, number of examples)
    layers_dims -- python list, containing the size of each layer
    learning_rate -- the learning rate, scalar.
    mini_batch_size -- the size of a mini batch
    beta -- Momentum hyperparameter
    beta1 -- Exponential decay hyperparameter for the past gradients estimates 
    beta2 -- Exponential decay hyperparameter for the past squared gradients estimates 
    epsilon -- hyperparameter preventing division by zero in Adam updates
    num_epochs -- number of epochs
    print_cost -- True to print the cost every 1000 epochs
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    L = len(layers_dims)             # number of layers in the neural networks
    costs = []                       # to keep track of the cost
    t = 0                            # initializing the counter required for Adam update
    seed = 10                        # For grading purposes, so that your "random" minibatches are the same as ours
    m = X.shape[1]                   # number of training examples
    
    # Initialize parameters
    parameters = initialize_parameters(layers_dims)

    # Initialize the optimizer
    if optimizer == "gd":
        pass # no initialization required for gradient descent
    elif optimizer == "momentum":
        v = initialize_velocity(parameters)
    elif optimizer == "adam":
        v, s = initialize_adam(parameters)
    
    # Optimization loop
    for i in range(num_epochs):
        
        # Define the random minibatches. We increment the seed to reshuffle differently the dataset after each epoch
        seed = seed + 1
        minibatches = random_mini_batches(X, Y, mini_batch_size, seed)
        cost_total = 0
        
        for minibatch in minibatches:

            # Select a minibatch
            (minibatch_X, minibatch_Y) = minibatch

            # Forward propagation
            a3, caches = forward_propagation(minibatch_X, parameters)

            # Compute cost and add to the cost total
            cost_total += compute_cost(a3, minibatch_Y)

            # Backward propagation
            grads = backward_propagation(minibatch_X, minibatch_Y, caches)

            # Update parameters
            if optimizer == "gd":
                parameters = update_parameters_with_gd(parameters, grads, learning_rate)
            elif optimizer == "momentum":
                parameters, v = update_parameters_with_momentum(parameters, grads, v, beta, learning_rate)
            elif optimizer == "adam":
                t = t + 1 # Adam counter
                parameters, v, s = update_parameters_with_adam(parameters, grads, v, s,
                                                               t, learning_rate, beta1, beta2,  epsilon)
        cost_avg = cost_total / m
        
        # Print the cost every 1000 epoch
        if print_cost and i % 1000 == 0:
            print ("Cost after epoch %i: %f" %(i, cost_avg))
        if print_cost and i % 100 == 0:
            costs.append(cost_avg)
                
    # plot the cost
    plt.plot(costs)
    plt.ylabel('cost')
    plt.xlabel('epochs (per 100)')
    plt.title("Learning rate = " + str(learning_rate))
    plt.show()
    return parameters

You will now run this 3 layer neural network with each of the 3 optimization methods.<br>
<br>
### 5.1 - Mini-batch Gradient descent<br>
<br>
Run the following code to see how the model does with mini-batch gradient descent.

In[24]:

train 3-layer model

In [ ]:
layers_dims = [train_X.shape[0], 5, 2, 1]
parameters = model(train_X, train_Y, layers_dims, optimizer = "gd")

Predict

In [ ]:
predictions = predict(train_X, train_Y, parameters)

Plot decision boundary

In [ ]:
plt.title("Model with Gradient Descent optimization")
axes = plt.gca()
axes.set_xlim([-1.5,2.5])
axes.set_ylim([-1,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

### 5.2 - Mini-batch gradient descent with momentum<br>
<br>
Run the following code to see how the model does with momentum. Because this example is relatively simple, the gains from using momemtum are small; but for more complex problems you might see bigger gains.

In[25]:

train 3-layer model

In [ ]:
layers_dims = [train_X.shape[0], 5, 2, 1]
parameters = model(train_X, train_Y, layers_dims, beta = 0.9, optimizer = "momentum")

Predict

In [ ]:
predictions = predict(train_X, train_Y, parameters)

Plot decision boundary

In [ ]:
plt.title("Model with Momentum optimization")
axes = plt.gca()
axes.set_xlim([-1.5,2.5])
axes.set_ylim([-1,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

### 5.3 - Mini-batch with Adam mode<br>
<br>
Run the following code to see how the model does with Adam.

In[ ]:

train 3-layer model

In [ ]:
layers_dims = [train_X.shape[0], 5, 2, 1]
parameters = model(train_X, train_Y, layers_dims, optimizer = "adam")

Predict

In [ ]:
predictions = predict(train_X, train_Y, parameters)

Plot decision boundary

In [ ]:
plt.title("Model with Adam optimization")
axes = plt.gca()
axes.set_xlim([-1.5,2.5])
axes.set_ylim([-1,1.5])
plot_decision_boundary(lambda x: predict_dec(parameters, x.T), train_X, train_Y)

### 5.4 - Summary<br>
<br>
<table> <br>
    <tr><br>
        <td><br>
        **optimization method**<br>
        </td><br>
        <td><br>
        **accuracy**<br>
        </td><br>
        <td><br>
        **cost shape**<br>
        </td><br>
<br>
    </tr><br>
        <td><br>
        Gradient descent<br>
        </td><br>
        <td><br>
        79.7%<br>
        </td><br>
        <td><br>
        oscillations<br>
        </td><br>
    <tr><br>
        <td><br>
        Momentum<br>
        </td><br>
        <td><br>
        79.7%<br>
        </td><br>
        <td><br>
        oscillations<br>
        </td><br>
    </tr><br>
    <tr><br>
        <td><br>
        Adam<br>
        </td><br>
        <td><br>
        94%<br>
        </td><br>
        <td><br>
        smoother<br>
        </td><br>
    </tr><br>
</table> <br>
<br>
Momentum usually helps, but given the small learning rate and the simplistic dataset, its impact is almost negligeable. Also, the huge oscillations you see in the cost come from the fact that some minibatches are more difficult thans others for the optimization algorithm.<br>
<br>
Adam on the other hand, clearly outperforms mini-batch gradient descent and Momentum. If you run the model for more epochs on this simple dataset, all three methods will lead to very good results. However, you've seen that Adam converges a lot faster.<br>
<br>
Some advantages of Adam include:<br>
- Relatively low memory requirements (though higher than gradient descent and gradient descent with momentum) <br>
- Usually works well even with little tuning of hyperparameters (except $\alpha$)

**References**:<br>
<br>
- Adam paper: https://arxiv.org/pdf/1412.6980.pdf